# Example Usage

In [ ]:
import chalc as ch
import numpy as np
import matplotlib.pyplot as plt
import h5py, warnings

For our data we sample 100 points on a circle with some noise and 100 points from inside the unit disk.

In [ ]:
np.random.seed(40)
num_points = 100
mean = [0, 0]
cov = np.eye(2)*0.01
x = np.array([[np.sin(2*np.pi*t), np.cos(2*np.pi*t)] for t in np.random.rand(num_points)]).T +\
    np.random.default_rng().multivariate_normal(mean, cov, num_points).T # points as columns
y = np.random.rand(num_points) * np.array([[np.sin(2*np.pi*t), np.cos(2*np.pi*t)] for t in np.random.rand(num_points)]).T
plt.scatter(x[0, :], x[1, :], s=10)
plt.scatter(y[0, :], y[1, :], s=10)
colours = [0]*100 + [1]*100
points = np.concatenate((x,y), axis=1)

We compute the chromatic alpha complex $K$ of the point cloud. $K$ has far fewer simplices than either the Cech or Vietoris-Rips complex, which each have $\displaystyle \binom{200}{2} = 19900$ edges and $\displaystyle \binom{200}{3} = 1313400$ 2-simplices. 

In [ ]:
K = ch.chromatic.alpha(points, colours)
print(f'{len(K.simplices[1])} 1-simplices')
print(f'{len(K.simplices[2])} 2-simplices')

We can compute the six-pack of persistent homology diagrams of the chromatic alpha complex, the chromatic Delaunay-Cech complex, and the chromatic Delaunay-Rips complex.

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    truncation = 1.0
    dgms_alpha = ch.sixpack.compute(points, colours, dom=[0,], method="chromatic alpha")
    fig_alpha, ax_alpha = ch.plotting.plot_sixpack(dgms_alpha, truncation, max_diagram_dim=2)
    fig_alpha.suptitle('Chromatic Alpha')
    dgms_delcech = ch.sixpack.compute(points, colours, dom=[0,], method="chromatic delcech")
    fig_delcech, ax_delcech = ch.plotting.plot_sixpack(dgms_delcech, truncation, max_diagram_dim=2)
    fig_delcech.suptitle('Chromatic Delaunay-Cech')
    dgms_delrips = ch.sixpack.compute(points, colours, dom=[0,], method="chromatic delrips")
    fig_delrips, ax_delrips = ch.plotting.plot_sixpack(dgms_delrips, truncation, max_diagram_dim=2)
    fig_delrips.suptitle('Chromatic Delaunay-Rips')

We can save the diagrams to file or load a diagram from file:

In [ ]:
with h5py.File('test.h5', 'w') as f:
    ch.sixpack.save_diagrams(dgms_alpha, f)

with h5py.File('test.h5', 'r') as f:
    dgms = ch.sixpack.load_diagrams(f)

We can visualise the 2-skeleton of the filtration for points in 2D:

In [ ]:
ani = ch.plotting.animate_filtration(K, points, np.linspace(0, 1.0, 45), 5)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    ani